In [ ]:
import sys
sys.path.append("../..")

In [ ]:
from src.marks_reader import *
from src.competition_data import *
from src.optimizer import *
from src.prediction_saver import *
from events_data import events
from datetime import datetime

In [ ]:
competition = CompetitionData(
    name="Olympic Games Tokyo 2020", 
    events_data=events, 
    start_date=datetime(2021, 7, 23),
)

event_folder = 'classified'
only = None

In [ ]:
data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

In [ ]:
results = load_json('results.json')

In [ ]:
def save_config(config: dict, event: str, sex: str, file: str = 'opt_results.json') -> None:
    optimal_results = load_json(file)

    
    if not event in optimal_results:
        optimal_results[event] = {}

    if not sex in optimal_results[event]:
        optimal_results[event][sex] = {}
    
    for key, value in config.items():
        optimal_results[event][sex][key] = value

    save_json(optimal_results, file)

In [ ]:
for event in competition.events:
    for sex in competition.get_event_data(event)['sex']:
        print(f"Optimizing {event} - {sex}...")
        
        smac = SMAC4BBOptimizer(1, calculate_error3)
        optimal_config = smac.optimize(data[event][sex], results, event, sex, competition)
        save_config(optimal_config, event, sex)